# <font color=green>Next Generation Sequencing (NGS) technologies for Whole Genome Sequencing (WGS) and de novo assembly (I)</font>

## Lecture Objectives:

- **Understand Illumina data in terms of read length, read depth and error rate**
- **Explore a pipeline for de novo genome assembly for Illumina data**

## Reading 

- <a href="https://www.melbournebioinformatics.org.au/tutorials/" target="_blank">Bioinformatics. Tutorials and Protocols</a>

- **The present and future of de novo whole-genome assembly by Sohn et al, Briefings in Bioinformatics. 2018; 19: 23**


## Illumina data

### What is Paired-End Sequencing?

Paired-end sequencing allows users to sequence both ends of a fragment and generate high-quality, alignable sequence data. Paired-end sequencing facilitates detection of genomic rearrangements and repetitive sequence elements, as well as gene fusions and novel transcripts.

In addition to producing twice the number of reads for the same time and effort in library preparation, sequences aligned as read pairs enable more accurate read alignment and the ability to detect insertion-deletion (indel) variants, which is not possible with single-read data.

<img src="paired-end-vs-single-read-seq-web-graphic.jpg" title="Paired-end sequencing" />

## Installing software needed for Part 1

For this course we will create a conda environment with all the software needed to produce the final assemblies of our strains.

Conda enviroments allow you to have all the software and packages needed in a simple environment. You don't need to have sudo access to create conda environments as they are installed in your home directory.

First we will create an environment with the name "course_assembly" and we will install in this environment all the software needed for this part of the course.

To create the conda environment you just need to type in you shell:

`conda create --name course`

With this an empty environment will be created with the name course_assembly

To active the conda environment:

`activate course_assebly`

Once the environment is activated in your shell you can install last version available of different packages using:

`conda install -c bioconda fastqc`

### Checking quality of paired end Illumina reads

#### FASTQC software


- <a href="http://www.bioinformatics.babraham.ac.uk/projects/fastqc/" target="_blank">FastQC documentation</a>

- <a href="http://manpages.ubuntu.com/manpages/bionic/man1/fastqc.1.html" target="_blank">FastQC Mannual pages</a>

FastQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis.

The Firt thing to do will be always to activate our environment where we have all the software we need installed

`source activate course`



In [15]:
%%bash 

#source activate course

# cd /opt/tljh/user/data/Enterococcus

# ls


In folder `fasq` we can find the Illumina paired-end sequences we will use for the assembly of the strain "58"

To run fastqc we just need to call the program followed by the two sets of fastq files (one per each direction of the paired end sequencing):



In [16]:
%%bash

#check location

#pwd

# we will storage our results in folder fastqc 

# mkdir fastqc 

#fastqc ./fastq/58_P3_S6_L001-4_R1_001.trim.fastq.gz ./fastq/58_P3_S6_L001-4_R2_001.trim.fastq.gz -t 8 -o fastqc

#### Fastp software

- <a href="https://github.com/OpenGene/fastp" target="_blank">Fastp Github website</a>


A tool designed to provide fast all-in-one preprocessing for FastQ files. This tool is developed in C++ with multithreading supported to afford high performance.

fastp supports both single-end (SE) and paired-end (PE) input/output. For PE data, you should also specify read2 input by -I or --in2, and specify read2 output by -O or --out2.


In [17]:
%%bash

#mkdir fastp

#fastp -i ./fastq/58_P3_S6_L001-4_R1_001.trim.fastq.gz -I ./fastq/58_P3_S6_L001-4_R2_001.trim.fastq.gz  -o ./fastp/58_P3_S6_L001-4_R1_001.fastq_trim.gz -O ./fastp/58_P3_S6_L001-4_R2_001.fastq_trim.gz -h --detect_adapter_for_pe -w 8


Quality of the trimmed reads can be observed again using FASTQC

In [18]:
%%bash

#mkdir fastqc_trimmed

#fastqc ./fastp/58_P3_S6_L001-4_R1_001.fastq_trim.gz ./fastp/58_P3_S6_L001-4_R2_001.fastq_trim.gz -t 8 -o fastqc_trimmed

#### Trimmomatic

- <a href="http://www.usadellab.org/cms/?page=trimmomatic" target="_blank">Trimmomatic website</a>

Trimmomatic is a fast, multithreaded command line tool that can be used to trim and crop Illumina (FASTQ) data as well as to remove adapters. These adapters can pose a real problem depending on the library preparation and downstream application.

There are two major modes of the program: Paired end mode and Single end mode. The paired end mode will maintain correspondence of read pairs and also use the additional information contained in paired reads to better find adapter or PCR primer fragments introduced by the library preparation process.
Trimmomatic works with FASTQ files (using phred + 33 or phred + 64 quality scores, depending on the Illumina pipeline used). Files compressed using either „gzip‟ or „bzip2‟ are supported, and are identified by use of „.gz‟ or „.bz2‟ file extensions.

The current trimming steps are:

   * ILLUMINACLIP: Cut adapter and other illumina-specific sequences from the read. SLIDINGWINDOW: Performs a sliding window trimming approach. It starts scanning at the 5‟ end and clips the read once the average quality within the window falls below a threshold.
   * MAXINFO: An adaptive quality trimmer which balances read length and error rate to maximise the value of each read
   * LEADING: Cut bases off the start of a read, if below a threshold quality
   * TRAILING: Cut bases off the end of a read, if below a threshold quality
   * CROP: Cut the read to a specified length by removing bases from the end HEADCROP: Cut the specified number of bases from the start of the read MINLEN: Drop the read if it is below a specified length
   * AVGQUAL: Drop the read if the average quality is below the specified level TOPHRED33: Convert quality scores to Phred-33
   * TOPHRED64: Convert quality scores to Phred-64
    

In [19]:
%%bash

#java -jar trimmomatic-0.39.jar PE input_forward.fq.gz input_reverse.fq.gz output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq.gz output_reverse_unpaired.fq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads *java -jar trimmomatic-0.39.jar PE input_forward.fq.gz input_reverse.fq.gz output_forward_paired.fq.gz output_forward_unpaired.fq.gz output_reverse_paired.fq.gz output_reverse_unpaired.fq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:15 TRAILING:20 SLIDINGWINDOW:5:20 MINLEN:50 -threads 8

This will perform the following:

* Remove adapters (ILLUMINACLIP:TruSeq3-PE.fa:2:30:10)

* Remove leading low quality or N bases (below quality 15) (LEADING:15)

* Remove trailing low quality or N bases (below quality 20) (TRAILING:20)

* Scan the read with a 4-base wide sliding window, cutting when the average quality per base drops below 15 (SLIDINGWINDOW:5:20)

* Drop reads below the 36 bases long (MINLEN:50)

* Use 8 threads

### Assembly of Illumina reads

#### SPAdes software

- <a href="https://github.com/ablab/spades" target="_blank">Spades Github</a>
- <a href="http://cab.spbu.ru/files/release3.13.0/manual.html" target="_blank">Spades manual</a>

SPAdes is not intended for larger genomes (e.g. mammalian size genomes). SPAdes is a very memory intensive program. In multithreaded mode (-t 16), you will want at least 500 Gigabytes if not 1000 Gigabytes of RAM.

SPAdes takes as input paired-end reads, mate-pairs and single (unpaired) reads in FASTA and FASTQ.SPAdes supports mate-pair only assembly.

**Read pair utilization**

While the use of paired reads and mate pairs is key to genome assembly, and not new, SPAdes utilizes so called paired DeBruin graphs to take the advantage of the paired end data. One of the key issues with paired DeBruin graphs is that the resulting genome assemblies do not tolerate variability in insert sizes: The initial formulation of paired DeBruijn graphs assumed constant distance between pairs of reads. In practice this distance is always variable. SPAdes performs k-bimer (these are k-mers derived from paired reads) adjustment to identify exact or nearly-exact distances for each k-bimer pair.

**Error correction**

Sequencing data contains a substantial number of sequencing errors that manifest themselves as deviations (bulges and non-connected components) within the assembly graph. One way to improve the graph before assembly it is to minimize the number of sequencing errors by performing error correction. SPAdes uses BayesHammer (Nikolenko et al. 2013) to correct the reads. Here is a brief summary of what it does:

SPAdes (or rather BayesHammer) counts k-mers in reads and computes k-mer statistics that take into account base quality values.

A Hamming graph is constructed in which k-mers are nodes. In this graph edges connect nodes (k-mers) if they differ from each other by a number of nucleotides up to a certain threshold (the Hamming distance). The graph is central to the error correction algorithm.

Then Bayesian subclustering is done on the graph from the previous step. For each k-mer we now know the center of its subcluster.

Solid k-mers are derived from cluster centers and are assumed to be error free.
Solid k-mers are mapped back to the reads.
Reads are corrected using solid k-mers.

**Code**

`source activate course`

`mkdir spades`

In [20]:
%%bash

#spades.py -1 ./fastp/58_P3_S6_L001-4_R1_001.trim.fastq.gz -2 ./fastp/58_P3_S6_L001-4_R2_001.trim.fastq.gz -o ./spades -t 8 --careful
